optuna is using beysian in backend

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.6 MB/s eta 0:00:00


In [2]:
import optuna

In [3]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Data set for Data science/Admission_Prediction.csv')

In [6]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [7]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [8]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [9]:
df["GRE Score"] = df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"] = df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"] = df["University Rating"].fillna(df["University Rating"].median())

In [10]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [11]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [12]:
X = df.drop(["Serial No.","Chance of Admit"],axis=1)

In [13]:
y = df["Chance of Admit"]

In [14]:
std_sca = StandardScaler()

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=25)

In [16]:
X_test.shape

(125, 7)

if you are using decision tree , then you can skip this standard part


In [17]:
X_train = std_sca.fit_transform(X_train)
X_test = std_sca.transform(X_test)


In [18]:
!nvidia-smi

Fri Aug 18 03:32:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [46]:
from xgboost.sklearn import XGBRegressor
def objective(trial,data=X,target=y):
  train_x,test_x,train_y,test_y = train_test_split(data,target,test_size=0.25,random_state=30)
  param = {
      "tree_method":"gpu_hist",
      'lambda' :trial.suggest_loguniform('lambda' ,1e-4,10.0),
      'alpha' : trial.suggest_loguniform('alpha', 1e-4, 18.0),
      'colsample_bytree': trial.suggest_categorical('colsample_bytree', [.1, .2, .3, .4, .5, .6, .7, .8,.9,1]),
      'subsample' : trial.suggest_categorical('subsample', [.1, .2, .3, .4, .5, .6, .7, .8,.9,1]),
      'learning_rate' : trial.suggest_categorical('learning_rate', [.00001, .0003, .008, .02, .01, 1,8]),
      'n_estimator' :30000,
      'max_depth' : trial.suggest_categorical ("max_depth", [3,4,5,6,7,8,9,10,11,12]),
      'random_state' : trial.suggest_categorical('random_state' ,[18,28,38,2000, 3454,243123]),
      'min_child_weight' : trial.suggest_int('min_child_weight',1,200)
      }
  Xgb_reg_model = XGBRegressor(**param)
  Xgb_reg_model.fit(train_x,train_y,eval_set =[(test_x,test_y)],verbose = True )
  predict_xgb = Xgb_reg_model.predict(test_x)
  mse = mean_squared_error(test_y,predict_xgb)
  return mse

In [47]:
find_params = optuna.create_study()
find_params.optimize(objective,n_trials =10)
find_params.best_trial.params

[I 2023-08-18 04:36:09,751] A new study created in memory with name: no-name-1c0cf552-3025-41e8-832f-3da901745893


[04:36:09] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.25574
[1]	validation_0-rmse:0.25392
[2]	validation_0-rmse:0.25220
[3]	validation_0-rmse:0.25043
[4]	validation_0-rmse:0.24867
[5]	validation_0-rmse:0.24695
[6]	validation_0-rmse:0.24535
[7]	validation_0-rmse:0.24365
[8]	validation_0-rmse:0.24194
[9]	validation_0-rmse:0.24031
[10]	validation_0-rmse:0.23864
[11]	validation_0-rmse:0.23703
[12]	validation_0-rmse:0.23548
[13]	validation_0-rmse:0.23398
[14]	validation_0-rmse:0.23255
[15]	validation_0-rmse:0.23114
[16]	validation_0-rmse:0.22981
[17]	validation_0-rmse:0.22842
[18]	validation_0-rmse:0.22699
[19]	validation_0-rmse:0.22560
[20]	validation_0-rmse:0.22422
[21]	validation_0-rmse:0.22278
[22]	validation_0-rmse:0.22137
[23]	validation_0-rmse:0.22005
[24]	validation_0-rmse:0.21876
[25]	validation_0-rmse:0.21748
[26]	validation_0-rmse:0.21622
[27]	validation_0-rmse:0.21500
[28]	validation_0-rmse:0.21385
[29]	validation_0-rmse

<ipython-input-46-512f7f09efdf>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-46-512f7f09efdf>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[41]	validation_0-rmse:0.19950
[42]	validation_0-rmse:0.19856
[43]	validation_0-rmse:0.19766
[44]	validation_0-rmse:0.19671
[45]	validation_0-rmse:0.19571
[46]	validation_0-rmse:0.19477
[47]	validation_0-rmse:0.19386
[48]	validation_0-rmse:0.19293
[49]	validation_0-rmse:0.19206
[50]	validation_0-rmse:0.19119
[51]	validation_0-rmse:0.19024
[52]	validation_0-rmse:0.18944
[53]	validation_0-rmse:0.18867
[54]	validation_0-rmse:0.18784
[55]	validation_0-rmse:0.18706
[56]	validation_0-rmse:0.18634
[57]	validation_0-rmse:0.18552
[58]	validation_0-rmse:0.18475
[59]	validation_0-rmse:0.18403
[60]	validation_0-rmse:0.18336
[61]	validation_0-rmse:0.18260
[62]	validation_0-rmse:0.18187
[63]	validation_0-rmse:0.18121
[64]	validation_0-rmse:0.18047
[65]	validation_0-rmse:0.17977
[66]	validation_0-rmse:0.17911
[67]	validation_0-rmse:0.17851
[68]	validation_0-rmse:0.17786
[69]	validation_0-rmse:0.17729
[70]	validation_0-rmse:0.17663
[71]	validation_0-rmse:0.17599
[72]	validation_0-rmse:0.17537
[73]	val

[I 2023-08-18 04:36:10,177] Trial 0 finished with value: 0.026342129599919883 and parameters: {'lambda': 0.00040004795586275906, 'alpha': 0.9316567210942347, 'colsample_bytree': 0.1, 'subsample': 0.5, 'learning_rate': 0.01, 'max_depth': 8, 'random_state': 18, 'min_child_weight': 137}. Best is trial 0 with value: 0.026342129599919883.


...

[04:36:14] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.25601
[1]	validation_0-rmse:0.25454
[2]	validation_0-rmse:0.25321
[3]	validation_0-rmse:0.25177
[4]	validation_0-rmse:0.25041
[5]	validation_0-rmse:0.24906
[6]	validation_0-rmse:0.24776
[7]	validation_0-rmse:0.24636
[8]	validation_0-rmse:0.24495
[9]	validation_0-rmse:0.24367
[10]	validation_0-rmse:0.24228
[11]	validation_0-rmse:0.24082
[12]	validation_0-rmse:0.23959
[13]	validation_0-rmse:0.23830
[14]	validation_0-rmse:0.23711
[15]	validation_0-rmse:0.23590
[16]	validation_0-rmse:0.23476
[17]	validation_0-rmse:0.23348
[18]	validation_0-rmse:0.23222
[19]	validation_0-rmse:0.23092
[20]	validation_0-rmse:0.22962
[21]	validation_0-rmse:0.22837
[22]	validation_0-rmse:0.22717
[23]	validation_0-rmse:0.22612
[24]	validation_0-rmse:0.22504
[25]	validation_0-rmse:0.22392
[26]	validation_0-rmse:0.22271
[27]	validation_0-rmse:0.22156
[28]	validation_0-rmse:0.22044
[29]	validation_0-rmse

<ipython-input-46-512f7f09efdf>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-46-512f7f09efdf>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[35]	validation_0-rmse:0.21338
[36]	validation_0-rmse:0.21244
[37]	validation_0-rmse:0.21149
[38]	validation_0-rmse:0.21058
[39]	validation_0-rmse:0.20966
[40]	validation_0-rmse:0.20853
[41]	validation_0-rmse:0.20764
[42]	validation_0-rmse:0.20682
[43]	validation_0-rmse:0.20588
[44]	validation_0-rmse:0.20502
[45]	validation_0-rmse:0.20402
[46]	validation_0-rmse:0.20312
[47]	validation_0-rmse:0.20227
[48]	validation_0-rmse:0.20133
[49]	validation_0-rmse:0.20055
[50]	validation_0-rmse:0.19975
[51]	validation_0-rmse:0.19891
[52]	validation_0-rmse:0.19808
[53]	validation_0-rmse:0.19735
[54]	validation_0-rmse:0.19651
[55]	validation_0-rmse:0.19579
[56]	validation_0-rmse:0.19509
[57]	validation_0-rmse:0.19431
[58]	validation_0-rmse:0.19355
[59]	validation_0-rmse:0.19274
[60]	validation_0-rmse:0.19208
[61]	validation_0-rmse:0.19130
[62]	validation_0-rmse:0.19057
[63]	validation_0-rmse:0.18992
[64]	validation_0-rmse:0.18917
[65]	validation_0-rmse:0.18816
[66]	validation_0-rmse:0.18754
[67]	val

[I 2023-08-18 04:36:14,538] Trial 9 finished with value: 0.02845955878020785 and parameters: {'lambda': 0.3011522890379999, 'alpha': 0.00010130540791952805, 'colsample_bytree': 0.8, 'subsample': 0.3, 'learning_rate': 0.008, 'max_depth': 11, 'random_state': 18, 'min_child_weight': 62}. Best is trial 1 with value: 0.0046246278985953935.


{'lambda': 0.15157782634941022,
 'alpha': 1.8996887005955956,
 'colsample_bytree': 1,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 5,
 'random_state': 28,
 'min_child_weight': 12}

In [48]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,2.634213e-02,2023-08-18 04:36:09.754862,2023-08-18 04:36:10.177049,0 days 00:00:00.422187,0.931657,0.1,0.000400,0.01000,8,137,18,0.5,COMPLETE
1,1,4.624628e-03,2023-08-18 04:36:10.180102,2023-08-18 04:36:10.630330,0 days 00:00:00.450228,1.899689,1.0,0.151578,1.00000,5,12,28,0.7,COMPLETE
2,2,6.383510e-02,2023-08-18 04:36:10.632677,2023-08-18 04:36:11.049464,0 days 00:00:00.416787,7.202456,0.8,0.000970,0.00030,7,178,3454,0.8,COMPLETE
3,3,6.283288e-02,2023-08-18 04:36:11.054183,2023-08-18 04:36:11.570337,0 days 00:00:00.516154,0.003584,0.9,0.000330,0.00030,10,39,18,0.5,COMPLETE
4,4,6.632720e-02,2023-08-18 04:36:11.575456,2023-08-18 04:36:12.023225,0 days 00:00:00.447769,0.397922,0.1,0.036726,0.00030,10,98,38,0.2,COMPLETE
5,5,6.623281e-02,2023-08-18 04:36:12.025086,2023-08-18 04:36:12.530434,0 days 00:00:00.505348,9.331647,0.8,0.000355,0.00001,3,2,2000,0.8,COMPLETE
6,6,7.159922e+73,2023-08-18 04:36:12.532344,2023-08-18 04:36:13.116030,0 days 00:00:00.583686,0.012625,0.7,6.501624,8.00000,8,9,243123,0.5,COMPLETE
7,7,6.621517e-02,2023-08-18 04:36:13.119282,2023-08-18 04:36:13.583350,0 days 00:00:00.464068,0.381415,0.3,0.000381,0.00001,9,135,38,1.0,COMPLETE
8,8,7.340606e-03,2023-08-18 04:36:13.587150,2023-08-18 04:36:14.076318,0 days 00:00:00.489168,0.010306,0.8,0.883196,0.02000,9,54,3454,0.5,COMPLETE
9,9,2.845956e-02,2023-08-18 04:36:14.079125,2023-08-18 04:36:14.537741,0 days 00:00:00.458616,0.000101,0.8,0.301152,0.00800,11,62,18,0.3,COMPLETE


In [49]:
optuna.visualization.plot_optimization_history(find_params)

In [50]:
optuna.visualization.plot_slice(find_params)

In [51]:
optuna.visualization.plot_contour(find_params, params = ['alpha', 'lambda'])

In [52]:
best_params={'lambda': 0.15157782634941022,
 'alpha': 1.8996887005955956,
 'colsample_bytree': 1,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 5,
 'random_state': 28,
 'min_child_weight': 12}

In [53]:
model = xgb.XGBRegressor(**best_params)

In [54]:
model.fit(X_train,y_train)

XGBRegressor(alpha=1.8996887005955956, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.15157782634941022,
             learning_rate=1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=12, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [55]:
y_pred = model.predict(X_test)

In [56]:
r2_score =[0,1]
from sklearn.metrics import r2_score

In [57]:
r2_score(y_test,y_pred)

0.7837738975377384

now try to implement Randomforestregressor on this data set

In [58]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2 = model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8190349883993683

**Homework **

xgboost intution--next class

difference between all the techniques

you need to take one more data regression and explore more and more about this optuna

take classificaiton dataset and check with the accracy with different different hyperparameter(optuna)

write atleast 5 differences between optuna and grid search and random search